In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install xgboost

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [2]:
df_train=pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_train.sample(3)

In [3]:
df_test=pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
df_test.sample(3)

In [4]:
y=df_train['SalePrice']

In [5]:
abs(df_train.corr()['SalePrice'].sort_values(ascending=False).head(30))

In [7]:
df1=pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [8]:
df2=pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [9]:
abs(df1.corr()['SalePrice'].sort_values(ascending=False).head(22))

In [10]:
df_train.shape,df_test.shape

In [11]:
df=df_train.append(df_test)

In [12]:
del df["SalePrice"]

In [13]:
df[df.select_dtypes(include=['object']).columns]

In [14]:
df[df.select_dtypes(include=[np.number]).columns]

In [15]:
empty=df.isnull().sum()
empty[empty>0]

In [16]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','PoolQC','MiscFeature','Alley','Fence','FireplaceQu'):
    df[col] = df[col].fillna('None')

In [17]:
df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

In [18]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars','BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    df[col] = df[col].fillna(0)

In [19]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    df[col] = df[col].fillna('None')

In [20]:
df["MasVnrType"] = df["MasVnrType"].fillna("None")
df["MasVnrArea"] = df["MasVnrArea"].fillna(0)

In [21]:
df['MSZoning']=df['MSZoning'].fillna(df['MSZoning'].mode()[0])

In [22]:
for col in ('Functional','Electrical','KitchenQual','Exterior1st','Exterior2nd','SaleType','Utilities'):
    df[col] = df[col].fillna(df[col].mode()[0])

In [23]:
df['MSSubClass'] = df['MSSubClass'].fillna("None")

In [24]:
df.isnull().sum()

In [25]:
empty=df.isnull().sum()
empty[empty>0]

In [26]:
df['MSSubClass'] = df['MSSubClass'].apply(str)
df['OverallCond'] = df['OverallCond'].astype(str)
df['YrSold'] = df['YrSold'].astype(str)
df['MoSold'] = df['MoSold'].astype(str)

In [27]:
df_gd=pd.get_dummies(df,drop_first=True)
df_gd.sample(20)

In [28]:
del df_gd['Id']

In [29]:
df_train.shape,df_test.shape, df_gd.shape

In [30]:
df12=df_gd[:1460]
df22=df_gd[1460:]

In [31]:
def algo_test(x,y):
        # define all models
        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        kn=KNeighborsRegressor()
        dt=DecisionTreeRegressor()
        xgb=XGBRegressor()
       
        
        algos=[L,R,Lass,E,ETR,GBR,kn,dt,xgb]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','Extra Tree','Gradient Boosting','KNeighborsRegressor','Decision Tree','XGBRegressor']
        
        x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.2,random_state=42)
        
        r_squared= []
        rmse= []
        mae= []
        
        # to create a dataframe to get accuracy rates
        result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
        
        
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r_squared.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
        
        # my accuracy and error rates in the table named result
        result.R_Squared=r_squared
        result.RMSE=rmse
        result.MAE=mae
        
       # sorts and returns the result table created according to its accuracy (r2_score)
        rtable=result.sort_values('R_Squared',ascending=False)
        return rtable

In [32]:
algo_test(df12,y)

In [33]:
df12_train, df12_test, y_train, y_test=train_test_split(df12,y,test_size=.2,random_state=42)
gb=GradientBoostingRegressor()
pred=gb.fit(df12_train,y_train).predict(df22)

In [34]:
predictions=pd.DataFrame()
predictions['Id']=df_test['Id']
predictions['SalePrice']=pred
predictions.to_csv('kaggle_predictions.csv',index=False)